In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("CountGerunds").getOrCreate()

In [ ]:
# perl -lne 'print join " ", m#\w+ing#g' HDFS.log > words_ing.txt

In [3]:
data = spark.read.text('/home/arthur/temp/LogPAI-data-uncompressed/words_ing.txt')

In [4]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer, HashingTF
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [5]:
data.printSchema()

root
 |-- value: string (nullable = true)



In [7]:
import pyspark.sql.functions as f

In [9]:
data.groupBy("value").count().show(4000, truncate=False)

+-----------------------+-------+
|value                  |count  |
+-----------------------+-------+
|trying                 |5545   |
|Starting               |7002   |
|Adding existing        |10     |
|Removing               |4      |
|Receiving              |1724696|
|Pending Pending Pending|47     |
|Changing               |65     |
|writing                |34     |
|serving                |356207 |
|terminating            |1706679|
|Deleting               |1402047|
|waiting                |103    |
|                       |5973190|
+-----------------------+-------+



In [10]:
logData=spark.read.csv('/home/arthur/temp/LogPAI-data-uncompressed/HDFS.log', sep=' ')

In [11]:
logData.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)



In [13]:
logData.show(truncate=False)

+------+------+---+----+-----------------------------+---------------+--------------------------+----------------------------------------------------------+------------------------+------------------------+-----------+--------------------+
|_c0   |_c1   |_c2|_c3 |_c4                          |_c5            |_c6                       |_c7                                                       |_c8                     |_c9                     |_c10       |_c11                |
+------+------+---+----+-----------------------------+---------------+--------------------------+----------------------------------------------------------+------------------------+------------------------+-----------+--------------------+
|081109|203518|143|INFO|dfs.DataNode$DataXceiver:    |Receiving      |block                     |blk_-1608999687919862906                                  |src:                    |/10.250.19.102:54106    |dest:      |/10.250.19.102:50010|
|081109|203518|35 |INFO|dfs.FSNamesystem

In [14]:
mergeColsUDF = f.udf(lambda _c5,_c6,_c7,_c8,_c9,_c10,_c11: _c5+_c6+_c7+_c8+_c9+_c10+_c11)
logData=logData.withColumn("logText", mergeColsUDF(f.col("_c5"),f.col("_c6"),f.col("_c7"),
                                                   f.col("_c8"),f.col("_c9"),f.col("_c10"),f.col("_c11")))

In [15]:
logData.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- logText: string (nullable = true)



In [17]:
logData.select("logText").show()

Py4JJavaError: An error occurred while calling o139.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 24.0 failed 1 times, most recent failure: Lost task 0.0 in stage 24.0 (TID 420, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/arthur/spark-2.3.2-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 253, in main
    process()
  File "/home/arthur/spark-2.3.2-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 248, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/arthur/spark-2.3.2-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 331, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/home/arthur/spark-2.3.2-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 140, in dump_stream
    for obj in iterator:
  File "/home/arthur/spark-2.3.2-b